<a href="https://colab.research.google.com/github/Shibu4064/EMNLP/blob/main/buetcsenlp_classification_model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install transformers[torch]
!pip install transformers[tf-cpu]
!pip install transformers[flax]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.6/225.6 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import multiprocessing
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import transformers
import torch.nn as nn
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoModelForMaskedLM, AutoModelForPreTraining, AutoModelForSequenceClassification, EarlyStoppingCallback
from transformers import AutoTokenizer, AutoConfig
from transformers import BertForMaskedLM, DistilBertForMaskedLM
from transformers import BertTokenizer, DistilBertTokenizer
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from tokenizers import BertWordPieceTokenizer
import gdown

In [3]:
import gdown

dataset_link = "https://drive.google.com/uc?id=1o4WpF71L5PNLBEl0HeTp13V35GZjEd5q"

# ami apatoto ekhane train dataset er drive link er id disi
# test dataset amar kase nai test dataset drive e upload diye share kore then oi id ta eikhane boshai dite hobe

gdown.download(dataset_link,"test.csv")



Downloading...
From: https://drive.google.com/uc?id=1o4WpF71L5PNLBEl0HeTp13V35GZjEd5q
To: /content/test.csv
100%|██████████| 607k/607k [00:00<00:00, 92.3MB/s]


'test.csv'

In [4]:
import gdown

checkpoint_link = "https://drive.google.com/uc?id=1-hNxX7itJapRTH9OzZc4l2IToK9RH11v"

gdown.download(checkpoint_link,"checkpoint.zip")

Downloading...
From: https://drive.google.com/uc?id=1-hNxX7itJapRTH9OzZc4l2IToK9RH11v
To: /content/checkpoint.zip
100%|██████████| 761M/761M [00:14<00:00, 50.8MB/s]


'checkpoint.zip'

In [5]:
!unzip checkpoint.zip

Archive:  checkpoint.zip
   creating: content/training_with_callbacks/checkpoint-9500/
  inflating: content/training_with_callbacks/checkpoint-9500/vocab.txt  
  inflating: content/training_with_callbacks/checkpoint-9500/optimizer.pt  
  inflating: content/training_with_callbacks/checkpoint-9500/special_tokens_map.json  
  inflating: content/training_with_callbacks/checkpoint-9500/scheduler.pt  
  inflating: content/training_with_callbacks/checkpoint-9500/rng_state.pth  
  inflating: content/training_with_callbacks/checkpoint-9500/trainer_state.json  
  inflating: content/training_with_callbacks/checkpoint-9500/tokenizer_config.json  
  inflating: content/training_with_callbacks/checkpoint-9500/tokenizer.json  
  inflating: content/training_with_callbacks/checkpoint-9500/config.json  
  inflating: content/training_with_callbacks/checkpoint-9500/training_args.bin  
  inflating: content/training_with_callbacks/checkpoint-9500/pytorch_model.bin  


In [6]:
from transformers import AutoTokenizer, AutoModel, AutoConfig

model_name = "csebuetnlp/banglabert"

tokenizer = AutoTokenizer.from_pretrained(model_name)

config = AutoConfig.from_pretrained("/content/content/training_with_callbacks/checkpoint-9500/config.json")

model = AutoModelForSequenceClassification.from_pretrained("/content/content/training_with_callbacks/checkpoint-9500/pytorch_model.bin",config=config)

model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [7]:
df_test = pd.read_csv("test.csv")

In [8]:
df_test.columns

Index(['text'], dtype='object')

In [9]:
df_test.rename(columns={"label": "label_column"}, inplace=True)

In [10]:
#test_dataest = Dataset.from_pandas(df_test.dropna())

In [11]:
def tokenize_function(example):
    example["text"] = tokenizer(example["text"], padding="max_length", truncation=True)

# Apply tokenization to the entire dataset
test_dataset = test_dataest.map(tokenize_function, batched=True)

Map:   0%|          | 0/2016 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.2 MB/s eta 0:00:00


In [15]:
from evaluate import evaluator
from datasets import load_dataset
task_evaluator = evaluator("text-classification")

results = task_evaluator.compute(
    model_or_pipeline=model,
    data=test_dataset,
    metric="f1",
    tokenizer=tokenizer,
    input_column="text",
    #label_column="label"
)

ValueError: ignored

In [ ]:
results